In [13]:
import xarray as xr
import sys
import pandas as pd
import numpy as np
import xesmf as xe
import warnings
import dask
import sys

from CASutils import calendar_utils as cal

warnings.filterwarnings('ignore')

In [14]:
import dask
dask.config.set({'distributed.dashboard.link': 'https://jupyterhub.ucar.edu/dav/user/{USER}/proxy/{port}/status'})

In [3]:
#cluster.close()

In [15]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client
dask.config.set({"distributed.scheduler.worker_saturation":"1.0"})

cluster = PBSCluster(
    cores = 1,
    memory = '30GB',
    processes = 1,
    queue = 'casper',
    local_directory = '$TMPDIR',
    resource_spec = 'select=1:ncpus=1:mem=20GB',
    project='P04010022',
    walltime='12:00:00',
    interface='ib0')

# scale up
cluster.scale(24)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

In [5]:
# get the workers going
#ncores = 36
#nmem = '460GB'
#nmem = str(int(365*ncores/36))+'GB'
#from dask_jobqueue import SLURMCluster
#from dask.distributed import Client
#cluster = SLURMCluster(cores=ncores,
#                     processes=ncores, memory=nmem,
#                     project='P04010022',
#                     walltime='12:00:00')
#cluster.scale(ncores)
#client = Client(cluster)

In [18]:
cluster

PBSCluster('tcp://10.12.206.54:37065', workers=16, threads=16, memory=343.60 GB)

In [7]:
#cluster.close()

In [19]:
# do this until you see you've got some workers
client

Client Scheduler: tcp://10.12.206.54:37065 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/islas/proxy/8787/status,Cluster Workers: 16 Cores: 16 Memory: 343.60 GB


In [20]:
# location of ERA5 data on RDA
filepath="/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.an.pl/"
# output location
outpath="/glade/scratch/islas/processed/era5/TEMdiags/"

In [21]:
ystart=2021 ; yend=2021 ; nyears=yend-ystart+1

In [22]:
# open up CESM data to get the output grid.
cesmdat = xr.open_dataset("/glade/campaign/cesm/collections/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/PHIS/f.e11.F1850C5CNTVSST.f09_f09.002.cam.h0.PHIS.040101-050012.nc")
grid_out = xr.Dataset({'lat': (['lat'], cesmdat.lat)}, {'lon': (['lon'], cesmdat.lon)})

In [23]:
%%time
reusewgt=False
wgtfile=outpath+"wgtfile.nc"
for iyear in range(ystart,yend+1,1):
    #for imon in range(1,13,1):
    for imon in range(1,13,1):
    #for imon in range(11,13,1):
        monstr=str(imon).zfill(2)
        print(str(iyear)+"-"+monstr)
        
        ds = xr.open_mfdataset(
        filepath + str(iyear)+monstr+ "/*_[u, v, w, t].*.nc",
        coords="minimal",
        join="override",
        decode_times=True,
        use_cftime=True,
        chunks={"time":24, "level":15},
        parallel=True,
        data_vars = "minimal",
        compat = "override")
        
        ds = ds.set_coords("utc_date")
        
        dayspermon = ds.time.dt.daysinmonth.data
        dayendstr = str(dayspermon[0])
        timeout = pd.date_range(
            start=str(iyear)+"-"+monstr+"-01",
            end=str(iyear)+"-"+monstr+"-"+dayendstr)
        
        ds = ds.loc[{"time": ds.time.dt.hour.isin([0,6,12,18])}]
             
        # convert to theta
        ds["T"] = ds.T * (ds.T.level / 1000.0)**(-2./7.)
        # convert Pa/s to m/s
        #!!!! Isla 08/30/21 - I think this still isn't right.  I think ds.W.level is in hPa instead of Pa
        #!!!! Isla 02/09/21 - Now I think this is right.  Added in *100 after dz.W.level
        g=9.81
        rgas=287.058
        ds["W"] = -1.*ds.W/ ( g * ((ds.W.level*100.)/(rgas*ds.T))) 
        ds["W"].attrs={'long_name':'Zonal-Mean vertical velocity','units':'m/s'}
        #!!!! I think this is wrong as I've already converted T to potential T here. (27th Feb 2023)
        
        
        
        regridder = xe.Regridder(ds.U, grid_out, 'bilinear', periodic=True, 
                                 reuse_weights=reusewgt, filename=wgtfile)
        reusewgt = True
        
        regridded = regridder(ds).persist()
        
        # this was causing issues before.  Either need to compute the zonal mean before
        # subtracting or do the persist on the regridded above.
        zonal_means = regridded.mean("lon")
        anomaly = regridded - zonal_means
        
        anomaly['uv'] = anomaly.U*anomaly.V
        anomaly['vt'] = anomaly.V*anomaly.T
        anomaly['uw'] = anomaly.U*anomaly.W
        
        zonal_means = zonal_means.merge(anomaly[['uv','vt','uw']].mean("lon"))
        
        temdiags = zonal_means.rename(
            {
                "uv":"UVzm",
                "vt":"VTHzm",
                "uw":"UWzm",
                "U":"Uzm",
                "V":"Vzm",
                "W":"Wzm",
                "T":"THzm",
            })
        
        temdiags.UVzm.attrs={'units':'m2/s2'}
        temdiags.VTHzm.attrs={'units':'Km/s'}
        temdiags.UWzm.attrs={'units':'m2/s2'}
        temdiags.Uzm.attrs={'units':'m/s'}
        temdiags.Vzm.attrs={'units':'m/s'}
        temdiags.Wzm.attrs={'units':'m/s'}
        temdiags.THzm.attrs={'units':'K'}
        
        temdiags = temdiags.groupby('time.dayofyear').mean()
        temdiags = temdiags.rename({'dayofyear':'time'})
        temdiags['time'] = timeout
        outfile = outpath+"fluxes_"+str(iyear)+"-"+monstr+".nc"
        # Since temdiags is small, it's better to load it and write in serial than in
        # parallel to netcdf.
        temdiags.load().to_netcdf(path=outfile)
        
        client.cancel(regridded) # deleted from distriubted RAM to recover memory
        ds.close()

2021-01
2021-02
2021-03
2021-04
2021-05
2021-06
2021-07
2021-08
2021-09
2021-10
2021-11
2021-12
CPU times: user 4min 7s, sys: 38.1 s, total: 4min 46s
Wall time: 1h 10min 7s


In [ ]:
temdiags.load().to_netcdf(path=outfile)

In [12]:
temdiags.time.encode['units'] = 'days since '+str(iyear)+'-'+str(imon).zfill(2)+'-'+str(imon).zfill(2)

AttributeError: 'DataArray' object has no attribute 'encode'

In [15]:
temdiags['time'] = timeout

In [16]:
print(temdiags.time)

<xarray.DataArray 'time' (time: 31)>
array(['1959-01-01T00:00:00.000000000', '1959-01-02T00:00:00.000000000',
       '1959-01-03T00:00:00.000000000', '1959-01-04T00:00:00.000000000',
       '1959-01-05T00:00:00.000000000', '1959-01-06T00:00:00.000000000',
       '1959-01-07T00:00:00.000000000', '1959-01-08T00:00:00.000000000',
       '1959-01-09T00:00:00.000000000', '1959-01-10T00:00:00.000000000',
       '1959-01-11T00:00:00.000000000', '1959-01-12T00:00:00.000000000',
       '1959-01-13T00:00:00.000000000', '1959-01-14T00:00:00.000000000',
       '1959-01-15T00:00:00.000000000', '1959-01-16T00:00:00.000000000',
       '1959-01-17T00:00:00.000000000', '1959-01-18T00:00:00.000000000',
       '1959-01-19T00:00:00.000000000', '1959-01-20T00:00:00.000000000',
       '1959-01-21T00:00:00.000000000', '1959-01-22T00:00:00.000000000',
       '1959-01-23T00:00:00.000000000', '1959-01-24T00:00:00.000000000',
       '1959-01-25T00:00:00.000000000', '1959-01-26T00:00:00.000000000',
       '1959-0

In [10]:
print(temdiags.time)

<xarray.DataArray 'time' (time: 31)>
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])
Coordinates:
  * time     (time) int64 1 2 3 4 5 6 7 8 9 10 ... 22 23 24 25 26 27 28 29 30 31


In [11]:
print(timeout)

DatetimeIndex(['1959-01-01', '1959-01-02', '1959-01-03', '1959-01-04',
               '1959-01-05', '1959-01-06', '1959-01-07', '1959-01-08',
               '1959-01-09', '1959-01-10', '1959-01-11', '1959-01-12',
               '1959-01-13', '1959-01-14', '1959-01-15', '1959-01-16',
               '1959-01-17', '1959-01-18', '1959-01-19', '1959-01-20',
               '1959-01-21', '1959-01-22', '1959-01-23', '1959-01-24',
               '1959-01-25', '1959-01-26', '1959-01-27', '1959-01-28',
               '1959-01-29', '1959-01-30', '1959-01-31'],
              dtype='datetime64[ns]', freq='D')
